# 1. Imports

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model # type: ignore
from tensorflow.keras import layers, models #type: ignore
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# 2. Load Data

In [ ]:
# Load raw OHLC data
df = pd.read_csv("../data/raw/merged-stock-data.csv")
print("Available columns:", df.columns.tolist())
print(f"Initial data shape: {df.shape}")

# Use ORIGINAL technical features (to match your trained model)
def add_technical_features(df):
    """Add original technical indicators (10 features total)"""
    df = df.copy()
    
    # Original features - keep exactly the same as training
    df['ma_5'] = df['close'].rolling(5).mean()
    df['ma_20'] = df['close'].rolling(20).mean()
    df['price_change'] = df['close'].pct_change()
    df['volatility'] = df['close'].rolling(10).std()
    df['hl_spread'] = (df['high'] - df['low']) / df['close']
    df['oc_spread'] = (df['close'] - df['open']) / df['open']
    
    return df.dropna()

# Use ORIGINAL feature columns (exactly 10 features)
feature_cols = [
    "open", "high", "low", "close",           # OHLC (4 features)
    "ma_5", "ma_20",                          # Moving averages (2 features)
    "price_change", "volatility",             # Price dynamics (2 features)
    "hl_spread", "oc_spread"                  # Spreads (2 features)
]                                             # Total: 10 features

print("Adding technical features...")
df = add_technical_features(df)
print(f"Features used: {len(feature_cols)} features (matches trained model)")
print(f"Data shape after feature engineering: {df.shape}")

x_raw = df[feature_cols].values

# Scale features
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x_raw)
print("Features scaled successfully")
print(f"Scaled features shape: {x_scaled.shape}")

# 3. Load Saved Pattern Recognition Model

In [ ]:
pattern_model = load_model("../models/candlestick_cnn_lstm.h5")

lookback = 20

x_seq = []
for i in range(lookback, len(x_scaled)):
    x_seq.append(x_scaled[i-lookback:i])

x_seq = np.array(x_seq)

patterns_preds = np.argmax(pattern_model.predict(x_seq), axis=1)
print("predicted pattern shape : ", patterns_preds.shape)

# 4. Create Trading Labels

In [ ]:
returns = df["close"].pct_change().values
returns = returns[lookback:]    

# Use exact percentiles for balanced classification
buy_threshold = np.percentile(returns, 70)   # Top 30% = Buy
sell_threshold = np.percentile(returns, 30)  # Bottom 30% = Sell

y_trading = np.ones(len(returns), dtype=int)  # Default to Hold (1)
y_trading[returns > buy_threshold] = 2       # Buy signal
y_trading[returns < sell_threshold] = 0      # Sell signal

print("Balanced Trading Labels Distribution:")
print(f"Sell (0): {np.sum(y_trading == 0)} ({np.sum(y_trading == 0)/len(y_trading)*100:.1f}%)")
print(f"Hold (1): {np.sum(y_trading == 1)} ({np.sum(y_trading == 1)/len(y_trading)*100:.1f}%)")
print(f"Buy (2): {np.sum(y_trading == 2)} ({np.sum(y_trading == 2)/len(y_trading)*100:.1f}%)")

print(f"\nReturn thresholds:")
print(f"Buy threshold (70th percentile): {buy_threshold*100:.3f}%")
print(f"Sell threshold (30th percentile): {sell_threshold*100:.3f}%")
print(f"Mean return: {np.mean(returns)*100:.3f}%")
print(f"Std return: {np.std(returns)*100:.3f}%")

# 5. Combine Features

In [ ]:
#converting pattern predictions to one-hot

num_patterns = np.max(patterns_preds) + 1
pattern_onehot = np.eye(num_patterns)[patterns_preds]

#combine OHLC sequences with patterns
#flatten sequences + pattern for MLP inputs
x_trading = x_seq.reshape(x_seq.shape[0], -1)
x_trading = np.hstack([x_trading, pattern_onehot])

print("Trading features shape : ", x_trading)

# 6. Train/Test Split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_trading, y_trading, test_size=0.2, shuffle=False)

print("Training set shape:", x_train.shape)
print("Test set shape:", x_test.shape)
print("Training labels distribution:", np.bincount(y_train))
print("Test labels distribution:", np.bincount(y_test))

# 7. Build Trading Signal Model (MLP)

In [ ]:
input_shape = x_train.shape[1]
num_classes = 3

inputs = layers.Input(shape=(input_shape,))

x = layers.Dense(128, activation="relu")(inputs)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.4)(x)

outputs = layers.Dense(num_classes, activation="softmax")(x)
trading_model = models.Model(inputs, outputs)

trading_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

trading_model.summary()

# 8. Train Model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights to handle imbalance
class_weights = compute_class_weight('balanced', 
                                   classes=np.unique(y_train), 
                                   y=y_train)
class_weight_dict = dict(enumerate(class_weights))
print("Class weights:", class_weight_dict)

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True, monitor='val_accuracy'),
    ReduceLROnPlateau(patience=3, factor=0.5, monitor='val_accuracy', verbose=1)
]

history = trading_model.fit(
    x_train, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=64,
    class_weight=class_weight_dict,
    callbacks=callbacks,
    verbose=1
)

# 9. Evaluate

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Basic evaluation
test_loss, test_accuracy = trading_model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")

# Detailed predictions
y_pred = trading_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes, 
                          target_names=['Sell', 'Hold', 'Buy']))

# Confusion matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Sell', 'Hold', 'Buy'],
            yticklabels=['Sell', 'Hold', 'Buy'])
plt.title('Trading Signal Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# 10. Plot Training History

In [ ]:
plt.plot(history.history["accuracy"], label="Training Accuracy")
plt.plot(history.history["val_accuracy"], label="Val Accuracy")
plt.xlabel("Epoches")
plt.ylabel("Accuracy")
plt.legend()
plt.show()